# 树回归

**主要内容**
> CART算法

> 回归树与模型树

> 树剪枝算法

> Python中GUI的使用

线性回归含有一些强大的方法，但这些方法创建的模型需要拟合所有的样本点（局部加权线性回归除外）。当数据拥有众多特征且特征之间关系十分复杂时，构建全局模型的想法就显得困难且笨拙。实际中很多问题都是非线性的，被=不可能使用全局线性模型来拟合任何数据。

一种可行的方法是**将数据集切分成很多易于建模的数据，然后利用线性回归技术来建模**。

## 1 复杂数据的局部建模

**分类与回归树(classification and regression tree, CART)**模型同样由特征选择、树的生成和剪枝组成，既可以用于分类也可用于回归。

前面利用决策树来进行分类。决策树不断将数据切分成小数据集，直到所有目标变量完全相同或者数据不能再切分为止。决策树是一种**贪心算法**，要在给定时间内做出最佳选择，但并不关心能否达到全局最优。

ID3的做法是每次选取当前最佳的特征来分割数据，并按照该特征的所有可能取值来切分。也就是说，如果一个特征有4个取值，那么数据将被切为4份。一旦按某种特征切分后，该特征在之后的算法执行过程中将不会再起作用，所有有观点认为**这种切分方式过于迅速**。另外一种方法是**二元切分发**，即每次把数据集切成两份。如果数据的某个特征值等于切分所要求的值，那么这些数据就进入树的左子树，反之进入右子树。

除切分过于迅速外，ID3还有一个问题，它**不能直接处理连续型特征**。只有事先将连续型数值转换成离散型，才能使用ID3算法。但这种**转换过程会破坏连续型变量的内在性质**。而使用二元切分法则易于对树构建过程进行调整以处理连续性特征。**具体做法**：如果特征值大于给定的给定值就走左子树，否则就走右子树。另外，二元切分法也节省了树的构建时间，这点意义不大，因为这些树的构建一般是离线完成，时间并非重点关注的因素。

<span style="color:red">树回归</span>
- **优点**：可以对复杂和非线性的数据建模
- **缺点**：结果不易理解
- **适用数据类型**：数值型和标称型数据

<span style="color:red">树回归的一般方法</span>
1. 收集数据：采用任意方法
2. 准备数据：需要数值型的数据，标称型数据应该映射成二值型数据
3. 分析数据：会出数据的二维可视化显示结果，以字典方式生成树
4. 训练算法：大部分时间都花在叶节点树模型的构建上
5. 测试算法：使用测试数据上的$R^2$值来分析模型结果
6. 使用算法：使用训练出的树做预测，预测结果还可以用来做很多事情

## 2 CART算法

CART是在给定输入随机变量$X$条件下输出随机变量$Y$的条件概率分布的学习方法。CART假设决策树是二叉树，内部节点特征的取值为“是”和“否”，左分支是取值为“是”的分支，右子树是取值为“否”的分支。这样的决策树等价于递归地二分每个特征，将输入空间即特征空间划分为有限个单元，并在这些单元上确定预测的概率分布，也就是在输入给定的条件下输出的条件概率分布。

<span style="color:red">CART算法由以下两步组成：</span>
1. **决策树的生成**：基于训练数据集生成决策树，生成的决策树要尽量大；
2. **决策树剪枝**：用验证数据集对生成的树进行剪枝并选择最优子树，这时用损失函数最小作为剪枝的标准.

决策树的生成就是递归地构建二叉决策树的过程。对回归树用**平方误差最小化准则**，对分类树用**基尼指数(Gini index)最小化准则**，进行特征先择，生成二叉树。

<span style="color:red">回归树的生成</span>

假设$X$与$Y$分别作为输入和输出变量，并且$Y$时连续变量，给定训练数据集$$D = \left\{(x_1,y_1), (x_2,y_2), \cdots, (x_N, y_N)\right\}$$

考虑如何生成回归树。

一个回归树对应着输入空间（即特征空间）的一个划分以及在划分的单元上的输出值。假设已将输入空间划分为$M$个单元$R_1, R_, \cdots, R_M$，并且在每个单元$R_m$上有一个固定的输出值$c_m$，于是回归树的模型可表示为
$$f(x) = \sum_{m=1}^Mc_mI(x\in R_m)$$

当输入空间的划分确定时，可以用**平方误差**$\sum_{x_i\in R_m}(y_i-f(x_i))^2$来表示回归树对于训练数据集的预测误差，用平方误差最小的准则求解每个单元上的最优输出值。易知，单元$R_m$上的$\hat{c_m}$的最优值$m$是$R_m$上的所有输出实例$x_i$对应的输出$y_i$的均值，即
$$\hat{c_m} = ave(y_i|x_i \in R_m)$$

问题是怎么**对输入空间进行划分**。这里**采用启发式的方法**，选择第$j$个变量$x^{(j)}$和它的取值$s$，作为切分变量(splitting variable)和切分点(splitting point)，并定义两个区域：
$$R_1(j,s) = \left\{x|x^{(j)}\leq s\right\}和R_2(j,s) = \left\{x|x^{(j)}>s\right\}$$

然后寻找最优切分变量$j$和最优切分点$s$.具体地，求解：

$$min_{j,s}\left[min_{c_1}\sum_{x_i\in R_1(j,s)}(y_i-c_1)^2 + min_{c_2}\sum_{x_i\in R_2(j,s)}(y_i-c_2)^2\right]$$

对固定输入变量$j$可以找到最优切分点$s$.
$$\hat{c_1} = ave\left(y_i|x_i\in R_1(j,s)\right)和\hat{c_1} = ave(y_i|x_i\in R_1(j,s))$$

遍历所有输入变量，找到最优地切分变量$j$，构成一个对$(j,s)$$.依此将输入空间划分成两个区域。接着，对每个区域重复上述过程，直到满足条件为止。这样生成一棵回归树。这样地回归树通常称为**最小二乘回归树(least squares regression tree)**，算法如下：

<span style="color:red">最小二乘树算法</span>

+ **输入**：训练数据集D
+ **输出**：回归树$f(x)$

在训练数据集所在的输入空间中，递归地将每个区域划分为两个子区域并决定每个子区域上的输出值，构建二叉决策树：
1. 选择最优切分变量$j$和切分点$s$，求解
$$min_{j,s}\left[min_{c_1}\sum_{x_i\in R_1(j,s)}(y_i-c_1)^2 + min_{c_2}\sum_{x_i\in R_2(j,s)}(y_i-c_2)^2\right]$$
遍历变量$j$，对固定的切分变量$j$扫描切分点$s$，选择使上式达到最小值的对$(j, s)$.
2. 用选定的对$(j, s)$划分区域并决定相应的输出值：
$$R_1(j,s) = \left\{x|x^{(j)}\leq s\right\}，R_2(j,s) = \left\{x|x^{(j)}>s\right\}$$
$$\hat{c_m} = \frac{1}{N_m}\sum_{x_i\in R_m(j,s)}y_i, x\in R_m, m=1,2$$
3. 继续对两个子区域调用步骤1，2，直到停止条件
4. 将输入空间划分为$M$个区域$R_1, R_2, \cdots, R_M$，生成决策树：
$$f(x) = \sum_{m=1}^M \hat{c_m}I(x\in R_m)$$

## 3 连续和离散型特征的树的构建